<a href="https://colab.research.google.com/github/JimGora/Asteroids/blob/main/Capstone_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Capstone 1**

by Jimmy Gora

####ML Zoomcamp 2023

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_text
%matplotlib inline

### Data Preparation

####Data loading

In [ ]:
url = "https://gitlab.com/mirsakhawathossain/pha-ml/-/raw/master/Dataset/dataset.csv"
df = pd.read_csv(url)

<ipython-input-2-f204903bd24a>:2: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [ ]:
#D
for i in df.columns:
  print(i)
  print()
  print(df[i].value_counts().sort_values(ascending=False))
  print()
  print()

####Preparing Data

In [ ]:
del_cols = ['id', 'spkid', 'full_name', 'pdes', 'name', 'prefix']
df = df.drop(columns=del_cols)

del_na = ['pha', 'sigma_per', 'ma', 'diameter', 'albedo', 'diameter_sigma', 'H']
for d in del_na:
  df = df.dropna(subset=[d])

map = {'Y': 1, 'N': 0}
df['neo'] = df['neo'].replace(map)
df['pha'] = df['pha'].replace(map)
df = df.sample(frac=1, random_state=12).reset_index(drop=True)

In [ ]:
#D
df['pha'].value_counts().sort_values(ascending=False)

0    130961
1       181
Name: pha, dtype: int64

####Training Model

In [ ]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=12)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=12)
df_full_train.reset_index(inplace=True, drop=True)
df_train.reset_index(inplace=True, drop=True)
df_val.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)
y_train = df_train.pha.values
y_val = df_val.pha.values
y_test = df_val.pha.values
del df_train['pha']
del df_val['pha']
del df_test['pha']

In [ ]:
ytr = y_train.sum()
yva = y_val.sum()
yte = y_test.sum()

print(ytr, yva, yte, ytr + yva + yte)

91 53 53 197


In [ ]:
train_dicts = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
features = dv.get_feature_names_out().tolist()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [ ]:
xgb_params = {
    'eta': 0.1,
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1}
model = xgb.train(xgb_params, dtrain, num_boost_round=175)

In [ ]:
y_pred = model.predict(dval)
roc_auc_score(y_val, y_pred)

0.9990888960649537

In [ ]:
0.9990888960649537

In [ ]:
dt = DecisionTreeClassifier(max_depth=6, min_samples_leaf=15)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=15)

In [ ]:
y_pred = dt.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_pred)

0.9999808985330073

In [ ]:
rf = RandomForestClassifier(n_estimators=210,
                            max_depth=10,
                            min_samples_leaf=3,
                            random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, n_estimators=210,
                       random_state=1)

In [ ]:
y_pred = rf.predict_proba(X_val)[:, 1]
roc_auc_score(y_val, y_pred)

0.9999726092171425